In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

# Data Setup & Functions

In [4]:
from mll_calc.mll_pred import format_XY, convert_g_to_mgUi

In [5]:
def logpdf_calc(row, test_sample, unc):
    y_sim = row[test_sample>0].values.tolist()
    std = row.multiply(unc)[test_sample>0].values.tolist()
    y_mes = test_sample[test_sample>0].values.tolist()
    logpdf_list = stats.norm.logpdf(y_sim, loc=y_mes, scale=std)
    return logpdf_list

In [6]:
def loop_sfco(XY, test, pred, unc, lbls, nonlbls):
    xy_cols = XY.columns.tolist()
    for col in nonlbls: xy_cols.remove(col)
    test = test[xy_cols]
    
    logpdf_df = pd.DataFrame()
    for test_idx, row in test.iterrows():
        test_sample = row.drop(lbls)
        test_answer = row[lbls]
        pred_row = pred.loc[pred['sim_idx'] == test_idx]
        pred_idx = pred_row['pred_idx'].values[0]
        train_row = XY.loc[pred_idx].drop(lbls+nonlbls)
        
        logpdf = logpdf_calc(train_row, test_sample, unc)
        mll = pred_row['MaxLogLL'].values[0]
        
        if round(mll, 4) != round(np.sum(logpdf), 4):
            print('mismatch')
            break
        
        logpdf = pd.Series(logpdf, index=train_row[test_sample>0].index, name=test_idx)
        if logpdf_df.empty:
            logpdf_df = pd.DataFrame(columns = test_sample.index.to_list())
        logpdf_df = logpdf_df.append(logpdf)
    return logpdf_df

### Train and Test DBs

In [7]:
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
nonlbls = ['AvgPowerDensity', 'ModDensity', 'UiWeight']

train_pkl = '~/sims_n_results/simupdates_aug2020/not-scaled_nuc29.pkl'
XY = format_XY(train_pkl)
XY = convert_g_to_mgUi(XY, lbls+nonlbls)

sfco_pkl = '~/sfcompo/format_clean/sfcompo_nuc29.pkl'
sfco = pd.read_pickle(sfco_pkl)

### LL Calc Results

In [8]:
sfcompo_results = '~/sims_n_results/simupdates_aug2020/sfco29/'

uncs = [0.05, 0.1, 0.15, 0.2]
job_dirs = ['Job' + str(i) + '_unc' + str(unc) for i, unc in enumerate(uncs)]
dfs = {}
for i, unc_job in enumerate(job_dirs):
    dfs['unc' + str(i)] = pd.read_csv(sfcompo_results + unc_job + '/' + unc_job + '.csv')

for dfname in list(dfs.keys()):
    df = dfs[dfname]
    df['Relative_Burnup_Error'] = df['Burnup_Error'] / df['Burnup']
    df['Relative_Enrichment_Error'] = df['Enrichment_Error'] / df['Enrichment']

# For ease of only viewing 5% case for now
preds = dfs['unc0']

### LogPDF (wrt nuclide) results

In [9]:
unc = 0.05
logpdf_df = loop_sfco(XY, sfco, preds, unc, lbls, nonlbls)

# Explore Errors

### Burnup

In [10]:
preds['Relative_Burnup_Error'].describe()
#nuc15 results for reference:
#count    505.000000
#mean       0.187539
#std        0.155398
#min        0.000408
#25%        0.058485
#50%        0.153846
#75%        0.298556
#max        1.503097

count    505.000000
mean       0.151887
std        0.195778
min        0.000031
25%        0.036301
50%        0.087295
75%        0.183161
max        1.440513
Name: Relative_Burnup_Error, dtype: float64

In [11]:
to_print = ['sim_idx', 'pred_idx', 'pred_CoolingTime', 'AvgPowerDensity', 
            'ReactorType', 'pred_ReactorType', 'ReactorType_Score', 
            'Enrichment', 'pred_Enrichment', 'Relative_Enrichment_Error', 
            'Burnup', 'pred_Burnup', 'Relative_Burnup_Error',
            'OrigenReactor', 'pred_OrigenReactor', 'MaxLogLL']
preds.loc[preds['Relative_Burnup_Error'] > 0.49, to_print]

,sim_idx,pred_idx,pred_CoolingTime,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
54,GAR-1|SA-13|E6|11,111254,1136.709042,10.0,bwr,bwr,True,2.410,5.230,1.170124,4200.0,8683.04,1.067390,Garigliano-1_BWR,abb8x8-1,-8.741122
55,GAR-1|SA-13|E6|13,50762,0.826167,10.0,bwr,bwr,True,2.410,4.570,0.896266,5580.0,9758.61,0.748855,Garigliano-1_BWR,ge7x7-0,11.544400
56,GAR-1|SA-13|E6|1,175085,109.336506,25.0,bwr,bwr,True,2.410,5.110,1.120332,6080.0,9304.01,0.530265,Garigliano-1_BWR,atrium10x10-9,10.778134
57,GAR-1|SA-13|E6|16,40811,829.867541,10.0,bwr,bwr,True,2.410,4.400,0.825726,6640.0,11427.93,0.721074,Garigliano-1_BWR,ge7x7-0,-8.061510
58,GAR-1|SA-13|E6|10,239103,8.087540,20.0,bwr,bwr,True,2.410,5.050,1.095436,8140.0,13912.97,0.709210,Garigliano-1_BWR,svea64-1,23.738365
60,GAR-1|SA-13|E6|7,59347,334.379204,25.0,bwr,bwr,True,2.410,4.570,0.896266,8320.0,13478.85,0.620054,Garigliano-1_BWR,ge7x7-0,23.983706
61,GAR-1|SA-13|E6|5,171906,242.633977,10.0,bwr,bwr,True,2.410,5.110,1.120332,8640.0,13992.11,0.619457,Garigliano-1_BWR,atrium10x10-9,3.973372
62,GAR-1|A-106|D4|1,41044,25.487227,10.0,bwr,bwr,True,2.100,4.400,1.095238,8850.0,17668.74,0.996468,Garigliano-1_BWR,ge7x7-0,-449.234902
63,GAR-1|A-106|E5|1,309124,29.215503,41.0,bwr,pwr,False,2.100,5.480,1.609524,8930.0,21793.78,1.440513,Garigliano-1_BWR,s18x18,-516.102948
64,GAR-1|A-106|C3|1,232501,0.462278,10.0,bwr,bwr,True,2.100,5.050,1.404762,9140.0,17118.70,0.872943,Garigliano-1_BWR,svea64-1,-491.230270


### Enrichment

In [12]:
preds['Relative_Enrichment_Error'].describe()
#nuc15 results for reference:
#count    505.000000
#mean       0.369961
#std        0.289978
#min        0.000000
#25%        0.119444
#50%        0.288235
#75%        0.611765
#max        2.853727

count    505.000000
mean       0.165083
std        0.248423
min        0.000000
25%        0.047923
50%        0.083004
75%        0.170455
max        1.750000
Name: Relative_Enrichment_Error, dtype: float64

In [13]:
preds.loc[preds['Relative_Enrichment_Error'] > 0.49, to_print]

,sim_idx,pred_idx,pred_CoolingTime,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
54,GAR-1|SA-13|E6|11,111254,1136.709042,10.0,bwr,bwr,True,2.41,5.23,1.170124,4200.0,8683.04,1.067390,Garigliano-1_BWR,abb8x8-1,-8.741122
55,GAR-1|SA-13|E6|13,50762,0.826167,10.0,bwr,bwr,True,2.41,4.57,0.896266,5580.0,9758.61,0.748855,Garigliano-1_BWR,ge7x7-0,11.544400
56,GAR-1|SA-13|E6|1,175085,109.336506,25.0,bwr,bwr,True,2.41,5.11,1.120332,6080.0,9304.01,0.530265,Garigliano-1_BWR,atrium10x10-9,10.778134
57,GAR-1|SA-13|E6|16,40811,829.867541,10.0,bwr,bwr,True,2.41,4.40,0.825726,6640.0,11427.93,0.721074,Garigliano-1_BWR,ge7x7-0,-8.061510
58,GAR-1|SA-13|E6|10,239103,8.087540,20.0,bwr,bwr,True,2.41,5.05,1.095436,8140.0,13912.97,0.709210,Garigliano-1_BWR,svea64-1,23.738365
59,GAR-1|SA-13|E6|3,165126,242.633977,25.0,bwr,bwr,True,2.41,4.23,0.755187,8260.0,11307.48,0.368944,Garigliano-1_BWR,atrium10x10-9,11.403446
60,GAR-1|SA-13|E6|7,59347,334.379204,25.0,bwr,bwr,True,2.41,4.57,0.896266,8320.0,13478.85,0.620054,Garigliano-1_BWR,ge7x7-0,23.983706
61,GAR-1|SA-13|E6|5,171906,242.633977,10.0,bwr,bwr,True,2.41,5.11,1.120332,8640.0,13992.11,0.619457,Garigliano-1_BWR,atrium10x10-9,3.973372
62,GAR-1|A-106|D4|1,41044,25.487227,10.0,bwr,bwr,True,2.10,4.40,1.095238,8850.0,17668.74,0.996468,Garigliano-1_BWR,ge7x7-0,-449.234902
63,GAR-1|A-106|E5|1,309124,29.215503,41.0,bwr,pwr,False,2.10,5.48,1.609524,8930.0,21793.78,1.440513,Garigliano-1_BWR,s18x18,-516.102948


# Max Log LL

In [14]:
print(dfs['unc0']['MaxLogLL'].describe())
print(dfs['unc3']['MaxLogLL'].describe())

count     505.000000
mean     -134.927067
std       246.620348
min     -2554.382504
25%      -136.777791
50%       -50.368502
75%       -18.304648
max        64.665774
Name: MaxLogLL, dtype: float64
count    490.000000
mean      10.182512
std       23.947474
min     -130.126567
25%       -3.514626
50%        3.204138
75%       21.907521
max       98.435790
Name: MaxLogLL, dtype: float64


# Nuclide Contributions to LL

In [15]:
nuc29 = ['am241', 'am242m', 'am243', 
         'cm242', 'cm244', 
         'cs134', 'cs137', 
         'eu154', 
         'nd143', 'nd144', 'nd145', 'nd146', 'nd148', 'nd150',
         'np237', 
         'pu238', 'pu239', 'pu240', 'pu241', 'pu242',
         'sm147', 'sm149', 'sm150', 'sm151', 'sm152',
         'u234', 'u235', 'u236', 'u238']

In [16]:
logpdf_df.sample(10)

,am241,am242m,am243,cm242,cm244,cs134,cs137,eu154,nd143,nd144,...,pu242,sm147,sm149,sm150,sm151,sm152,u234,u235,u236,u238
MON-1|MTB099|A1|S4,NaN,NaN,NaN,5.701263,NaN,NaN,NaN,NaN,NaN,NaN,...,-8.450708,NaN,NaN,NaN,NaN,NaN,-63.733032,-0.926910,1.154729,-4.796544
TAK-3|NT3G23|SF95|5,1.720071,8.762390,0.967821,6.611183,2.235671,2.702204,1.782535,5.840749,2.056807,2.214982,...,-2.604896,NaN,NaN,NaN,NaN,NaN,-3.870009,-0.733959,0.516318,-4.768950
GAR-1|A-106|E5|1,6.209387,NaN,-66.975933,6.125592,-1.230478,NaN,NaN,NaN,NaN,NaN,...,-73.914040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TVC-1|509-032|E11|4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.804107,NaN,NaN,NaN,NaN,NaN,-2.545652,-0.972570,0.693521,-4.787017
YNK-1|E6|NW-A1|T-162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.500148,NaN,NaN,NaN,NaN,NaN,-37.003867,-3.585351,-0.147187,-4.780078
MIH-3|JPNNM3SFA3|H13|87C07,0.595275,8.369913,4.287629,10.788233,0.831596,2.788992,0.876751,4.072577,2.152434,0.476977,...,1.708635,NaN,NaN,NaN,NaN,NaN,NaN,-1.185216,NaN,-4.778754
YNK-1|F4|C-F6|G-117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.077542,NaN,NaN,NaN,NaN,NaN,-34.473449,-1.304830,1.046660,-4.785268
KAL-1|623|312|28,NaN,NaN,7.907200,NaN,9.139225,NaN,NaN,NaN,NaN,NaN,...,1.764155,NaN,NaN,NaN,NaN,NaN,-20.860849,-3.054663,-0.053072,-4.783016
VAN-2|EF05|WZR0058|E58-793,-47.640097,NaN,-3.266326,NaN,-43.757355,3.936822,-2.828296,3.090053,-1.176763,-3.229547,...,0.138483,3.208970,6.564944,2.018584,3.989921,0.208491,3.175965,-4.955366,-7.924703,NaN
GOS-1|1701|16B05|GU3' (Cycle 18),2.009930,9.069915,-3.502505,8.815362,2.110662,4.166690,1.483594,4.214821,1.889224,1.231874,...,0.838122,3.666432,-0.504183,2.090607,5.624280,3.528672,-2.750877,0.171821,0.278416,-4.753364


In [17]:
for nuc in nuc29:
    desc = logpdf_df[nuc].describe()
    print(desc[['count', 'mean', 'std', 'min', 'max']])

count    237.000000
mean     -43.111645
std       97.121443
min     -794.506648
max        7.443631
Name: am241, dtype: float64
count    110.000000
mean     -28.865569
std       54.989217
min     -186.267401
max       13.108325
Name: am242m, dtype: float64
count    203.000000
mean      -7.205421
std       17.722737
min     -101.038273
max        9.315329
Name: am243, dtype: float64
count    214.000000
mean      -7.635268
std       36.916614
min     -302.590970
max       18.925809
Name: cm242, dtype: float64
count    269.000000
mean     -14.969502
std       38.596167
min     -175.276748
max       17.242042
Name: cm244, dtype: float64
count    113.000000
mean     -29.159047
std       56.604572
min     -180.668807
max        9.725492
Name: cs134, dtype: float64
count    185.000000
mean      -4.224876
std       11.265662
min      -70.572595
max        4.037617
Name: cs137, dtype: float64
count    100.000000
mean     -23.055338
std       42.402957
min     -166.955348
max        8.000693
Nam

# Look at ratio values for a single prediction

In [18]:
sim_id = 'VAN-2|EF05|WZR0058|E58-88'
pred_id = 43225
preds.loc[preds['sim_idx']==sim_id, to_print]

,sim_idx,pred_idx,pred_CoolingTime,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
418,VAN-2|EF05|WZR0058|E58-88,261683,1945.4721,25.0,pwr,pwr,True,4.5,5.39,0.197778,43520.0,40607.34,0.066927,Vandellos-2_PWR,ce14x14,-53.440264


In [19]:
y_mes = sfco.loc[sfco.index == sim_id].squeeze().drop(lbls)
y_sim = XY.loc[XY.index == pred_id].squeeze().drop(lbls+nonlbls)
y_logpdf = logpdf_df.loc[logpdf_df.index == sim_id].squeeze()
df = pd.DataFrame([y_mes, y_sim])
df.loc['Abs Diff'] = np.abs(df.loc[pred_id] - df.loc[sim_id])
df.loc['% Diff'] = df.loc['Abs Diff'] * 100 / df.loc[sim_id]
df.loc['LogPDF'] = y_logpdf

In [21]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df

,am241,am242m,am243,cm242,cm244,cs134,cs137,eu154,nd143,nd144,nd145,nd146,nd148,nd150,np237,pu238,pu239,pu240,pu241,pu242,sm147,sm149,sm150,sm151,sm152,u234,u235,u236,u238
VAN-2|EF05|WZR0058|E58-88,0.252000,0.000000,0.090100,0.000000,0.016500,0.018400,1.430000,0.016600,1.020000,1.650000,0.842000,0.852000,0.484000,0.235000,0.610000,0.268000,6.140000,2.400000,1.130000,0.556000,0.275000,0.003390,0.331000,0.014400,0.125000,0.237000,12.900000,5.510000,0.0
43225,0.356800,0.000905,0.084320,0.000005,0.021330,0.015460,1.154000,0.014720,0.925500,1.491000,0.774400,0.770200,0.408100,0.189500,0.482600,0.183700,4.826000,1.581000,0.938900,0.472900,0.277700,0.002967,0.307400,0.012120,0.114600,0.230200,13.390000,5.313000,934.0
Abs Diff,0.104800,0.000905,0.005780,0.000005,0.004830,0.002940,0.276000,0.001880,0.094500,0.159000,0.067600,0.081800,0.075900,0.045500,0.127400,0.084300,1.314000,0.819000,0.191100,0.083100,0.002700,0.000423,0.023600,0.002280,0.010400,0.006800,0.490000,0.197000,934.0
% Diff,41.587302,inf,6.415094,inf,29.272727,15.978261,19.300699,11.325301,9.264706,9.636364,8.028504,9.600939,15.681818,19.361702,20.885246,31.455224,21.400651,34.125000,16.911504,14.946043,0.981818,12.477876,7.129909,15.833333,8.320000,2.869198,3.798450,3.575318,inf
LogPDF,-23.891769,NaN,3.952035,NaN,-7.684340,1.498427,-0.087854,2.777426,1.023434,1.517373,2.040362,2.205196,2.022781,0.494180,2.546734,-4.827878,-0.356085,-34.313905,1.952183,-6.366853,2.324237,5.403919,2.741665,5.695389,3.777571,3.335053,-17.739232,-3.480312,NaN
